In [2]:
import pandas as pd
import numpy as np
import json
import ast
import nltk

# File to analyze: user_reviews.json

In [3]:
reviews = []
# open file json
with open('data/user_reviews.json', 'r', encoding='utf-8') as f:
    # loop for insert data into list
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))
# create data frame with data
df_user_reviews = pd.DataFrame(reviews)

df_user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [4]:
dfreviewsOpen = df_user_reviews.explode('reviews', ignore_index=True) 
df_user_reviews = pd.concat([dfreviewsOpen.drop(['reviews'], axis=1), dfreviewsOpen['reviews'].apply(pd.Series)], axis=1)
# 13.8s

In [5]:
df_user_reviews = df_user_reviews.drop(df_user_reviews.columns[-1], axis=1)
df_user_reviews.drop(['user_url', 'funny', 'last_edited'], axis=1, inplace=True)

In [6]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download(['vader_lexicon', 'stopwords', 'punkt', 'names'])

analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /home/p/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /home/p/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/p/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package names to /home/p/nltk_data...
[nltk_data]   Package names is already up-to-date!


In [7]:
df_user_reviews['sentiment_score'] = df_user_reviews['review'].map(
    lambda x: int(round(analyzer.polarity_scores(x)['compound'],0) + 1) if isinstance(x, str) else 1
)

In [8]:
# create column year that need for answer 
df_user_reviews['posted year'] = df_user_reviews['posted'].str.extract(r'(\d{4})')

# remove column posted
df_user_reviews.drop('posted' , axis = 1, inplace = True)

In [9]:
df_user_reviews.dropna(subset=['item_id'], inplace=True)

In [10]:
df_user_reviews

,user_id,item_id,helpful,recommend,review,sentiment_score,posted year
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2,2011
1,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,1,2011
2,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2,2011
3,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2,2014
4,js41637,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2,2013
...,...,...,...,...,...,...,...
59328,76561198312638244,70,No ratings yet,True,a must have classic from steam definitely wort...,2,NaN
59329,76561198312638244,362890,No ratings yet,True,this game is a perfect remake of the original ...,2,NaN
59330,LydiaMorley,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,2,NaN
59331,LydiaMorley,730,No ratings yet,True,:D,2,NaN


In [11]:
df_user_reviews.to_csv('df_user_reviews.csv', index=False)